# Python for Finance - part 9

#### 금융분석과 관련된 기본적인 내용을 정리하였습니다. 파이썬을 활용한 금융분석 (이브 힐피시 지음)을 참고하였습니다.

## Chapter 16 금융 모형 시뮬레이션
#### 기하 브라운 운동 : 옵션 가격 계산에 대한 블랙과 숄즈의 연구에서 처음 소개 / 옵션이나 기타 파생상품 가치 평가 목적으로 가장 널리 사용되는 확률 과정
#### 점프 확산 : 기하 브라운 운동 모형에 로그 정규분포에 따르는 점프 요인을 추가 / 큰 가격 점프의 가능성이 있는  out-of the money 옵션을 고려할 수 있게 됨 
#### 제곱근 확산 : 이자율이나 변동성과 같이 평균-회귀 특성을 보이는 변수의 모형에 활용 / 항상 양의 값

### 16.1 난수 생성

In [3]:
#### 표준정규분포 난수를 생성하는 함수
import numpy as np
def sn_random_numbers(shape, antithetic=True, moment_matching=True, fixed_seed=False):
    """ shape 인수와 같은 shape를 가지면서 표준 정규분포를 따르는 (의사) 난수 """
    
    if fixed_seed:
        np.random.seed(1000)
    if antithetic:
        ran = np.random.standard_normal((shape[0], shape[1], shape[2] / 2))
        ran = np.concatenate((ran, -ran), axis=2)
        
    else:
        ran = np.random.standard_normal(shape)
    
    if moment_matching:
        ran = ran - np.mean(ran)
        ran = ran / np.std(ran)
        
    if shape[0] ==1:
        return ran[0]
    else:
        return ran
    
snrn = sn_random_numbers((2,2,2), antithetic=False, moment_matching=False, fixed_seed=True)
snrn

array([[[-0.8044583 ,  0.32093155],
        [-0.02548288,  0.64432383]],

       [[-0.30079667,  0.38947455],
        [-0.1074373 , -0.47998308]]])

In [5]:
snrn_mm = sn_random_numbers((2,3,2), antithetic=False, moment_matching=True, fixed_seed=True)
snrn_mm

array([[[-1.47414161,  0.67072537],
        [ 0.01049828,  1.28707482],
        [-0.51421897,  0.80136066]],

       [[-0.14569767, -0.85572818],
        [ 1.19313679, -0.82653845],
        [ 1.3308292 , -1.47730025]]])

In [6]:
snrn_mm.mean()

3.700743415417188e-17

In [7]:
snrn_mm.std()

1.0

### 16.2 일반적인 시뮬레이션 클래스

In [8]:
import numpy as np
import pandas as pd

class simulation_class(object):
    """ 시뮬레이션 클래스의 베이스 메서드 제공 """ 
    
    def __init__(self, name, mar_env, corr):
        try:
            self.name = name
            self.pricing_date = mar_env.pricing_date
            self.initial_value = mar_env.get_constant('initial_value')
            self.volatility = mar_env.get_constant('volatility')
            self.final_date = mar_env.get_constant('final_date')
            self.currency = mar_env.get_constant('currency')
            self.frequency = mar_env.get_constant('frequency')
            self.paths = mar_env.get_constant('paths')
            self.discount_curve = mar_env.get_constant('discount_curve')
            
            try:
                self.time_grid = mar_env.get_list('time_grid')
            except:
                self.time_grid = None
            
            try: 
                self.special_dates = mar_env.get_list('special_dates')
            except:
                self.special_dates = []
                
            self.instrument_value = None
            self.correlated = corr
            if corr:
                self.cholesky_matrix = mar_env.get_list('cholesky_matrix')
                self.rn_set = mar_env.get_list('rn_set')[self.name]
                self.random_numbers = mar_env.get_list('random_numbers')
                
        except:
            print('Error parsing market environment')
                
    def generate_time_grid(self):
        start = self.pricing_date
        end = self.final_date
        time_grid = pd.date_range(start=start, end=end, freq=self.frequency)
        time_grid = list(time_grid)
        
        if start not in time_grid:
            time_grid.insert(0, start)
        
        if end not in time_grid:
            time_grid.append(end)
        
        if len(self.special_dates) >0:
            time_grid.extend(self.special_dates)
            time_grid = list(set(time_grid))
            time_grid.sort()
        self.time_grid = np.array(time_grid)
        
    def get_instrument_values(self, fixed_seed=True):
        if self.initial_value is None:
            self.generate_paths(fixed_seed=fixed_seed, day_count=365.)
        elif fixed_seed is False:
            self.generate_paths(fixed_seed=fixed_seed, day_count=365.)
            
        return self.instrument_value

C:\Users\QOOK\AppData\Local\Programs\Python\Python36-32\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 112 from C header, got 124 from PyObject
  return f(*args, **kwds)


### 16.3 기하 브라운 운동 모형

In [10]:
class geometric_brownian_motion(simulation_class):
    """ 블랙-숄즈-머튼의 기하 브라운 운동 모형에 따른 시뮬레이션 경로를 생성하는 클래스 """
    
    def __init__(self, name, mar_env, corr=False):
        super(geometric_brownian_motion, self).__init__(name, mar_env, corr)
        
    def update(self, initial_value=None, volatility = None, final_date = None):
        if initial_value is not None:
            self.initial_value = initial_value
        if volatility is not None:
            self.volatility = volatility
        if final_date is not None:
            self.final_date = final_date
        self.instrument_values = None
        
    def generate_paths(self, fixed_seed = False, day_count=365.):
        if self.time_grid is None:
            self.generate_time_grid()
        M = len(self.time_grid)
        I = self.paths
        paths = np.zeros((M,I))
        paths[0] = self.initial_value
        if not self.correlated:
            rand = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)
        else:
            rand = self.random_numbers
        
        short_rate = self.discount_curve.short_rate
        
        for t in range(1, range(self.time_grid)):
            if not self.correlated:
                ran = rand[t]
            else:
                ran = np.dot(self.cholesky_matrix, rand[:, t, :])
                ran = ran[self.rn_set]
            dt = (self.time_grid[t] - self.time_grid[t-1]).days / day_count
            paths[t] = paths[t-1] * np.exp((short_rate - 0.5 * self.volatility **2) *dt + self.volatility * np.sqrt(dt) * ran)
            
        self.instrument_values = paths

### 16.4 점프 확산 모형

In [16]:
class jump_diffusion(simulation_class):
    """ 머튼의 점프 확산 모형에 따른 시뮬레이션 경롤르 생성하기 위한 클래스 """
    
    def __init__(self, name, mar_env, corr=False):
        super(jump_diffusion, self).__init__(name, mar_env, corr)
        try:
            self.lamb = mar_env.get_constant('lambda')
            self.mu = mar_env.get_constant('mu')
            self.delt = mar_env.get_constant('delta')
            
        except:
            print('Error parsing market environment')
            
    def update(self, initial_value=None, volatility = None, lamb=None, mu=None, delta=None, final_date = None):
        if initial_value is not None:
            self.initial_value = initial_value
        if volatility is not None:
            self.volatility = volatility
        if lamb is not None:
            self.lamb = lamb
        if delta is not None:
            self.delta = delta
        if mu is not None:
            self.mu = mu
        if final_date is not None:
            self.final_date = final_date
        self.instrument_values = None

    def generate_paths(self, fixed_seed = False, day_count=365.):
        if self.time_grid is None:
            self.generate_time_grid()
        M = len(self.time_grid)
        I = self.paths
        paths = np.zeros((M,I))
        paths[0] = self.initial_value
        if not self.correlated:
            sn1 = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)
        else:
            sn1 = self.random_numbers
            
        sn2 = sn_random_numbers((1,M,I), fixed_seed=fixed_seed)
        rj = self.lamb * (np.exp(self.mu + .5 * self.delt ** 2) -1)
        short_rate = self.discount_curve.short_rate
        
        for t in range(1, len(self.time_grid)):
            if self.correlated is False:
                ran = sn1[t]
            else:
                ran = np.dot(self.cholesky_matrix, rand[:, t, :])
                ran = ran[self.rn_set]
            dt = (self.time_grid[t] - self.time_grid[t-1]).days / day_count
            poi = np.random.poisson(self.lamb * dt, I)
            paths[t] = paths[t-1] * (np.exp((short_rate - rj - .5 * self.volatility **2) *dt + self.volatility * np.sqrt(dt) * ran) + (np.exp(self.mu + self.delt * sn2[t]) -1) * poi)
                                     
        self.instrument_values = paths

### 16.5 제곱근 확산 모형

In [17]:
class square_root_diffusion(simulation_class):
    """ 콕스-잉거솔-로스의 제곱근 확산 모형에 기반한 시뮬레이션 경로를 생성하기 위한 클래스 """
    def __init__(self, name, mar_env, corr=False):
        super(square_root_diffusion, self).__init__(name, mar_env, corr)
        try:
            self.kappa = mar_env.get_constant('kappa')
            self.theta = mar_env.get_constant('theta')
            
        except:
            print('Error parsing market environment')
            
    def update(self, initial_value=None, volatility = None, kappa=None, theta=None, final_date = None):
        if initial_value is not None:
            self.initial_value = initial_value
        if volatility is not None:
            self.volatility = volatility
        if kappa is not None:
            self.lamb = lamb
        if dethetalta is not None:
            self.delta = delta
        if final_date is not None:
            self.final_date = final_date
        self.instrument_values = None

    def generate_paths(self, fixed_seed = False, day_count=365.):
        if self.time_grid is None:
            self.generate_time_grid()
        M = len(self.time_grid)
        I = self.paths
        paths = np.zeros((M,I))
        paths_ = np.zeros_like(paths)
        paths[0] = self.initial_value
        paths_[0] = self.initial_value
        if not self.correlated:
            rand = sn_random_numbers((1, M, I), fixed_seed=fixed_seed)
        else:
            rand = self.random_numbers
    
        for t in range(1, len(self.time_grid)):
            dt = (self.time_grid[t] - self.time_grid[t-1]).days / day_count
            if self.correlated is False:
                ran = rand[t]
            else:
                ran = np.dot(self.cholesky_matrix, rand[:, t, :])
                ran = ran[self.rn_set]
                
            paths_[t] = (paths[t-1] + self.kappa * (self.theta - np.maximum(0, paths_[t-1, :])) * dt + np.sqrt(np.maximum(0, paths_[t-1, :])) * self.volatility * np.sqrt(dt) * ran) 
            paths[t] = np.maximum(0, paths_[t])                         
        self.instrument_values = paths